# Project-3: Finsights Grey - RAG for Effective Information Retrieval


## Business Use Case

**Problem Statement:**

Finsights Grey Inc. is an innovative financial technology firm that specializes in providing advanced analytics and insights for investment management and financial planning. The company handles an extensive collection of 10-K reports from various industry players, which contain detailed information about financial performance, risk factors, market trends, and strategic initiatives. Despite the richness of these documents, Finsights Grey's financial analysts struggle with extracting actionable insights efficiently in a short span due to the manual and labor-intensive nature of the analysis. Going through the document to find the exact information needed at the moment takes too long. This bottleneck hampers the company's ability to deliver timely and accurate recommendations to its clients. To overcome these challenges, Finsights Grey Inc. aims to implement a Retrieval-Augmented Generation (RAG) model to automate the extraction, summarization, and analysis of information from the 10-K reports, thereby enhancing the accuracy and speed of their investment insights.

**Objective:**

As a Gen AI Data Scientist hired by Finsights Grey Inc., the objective is to develop an advanced RAG-based system to streamline the extraction and analysis of key information from 10-K reports. You are asked to deploy a Gradio app on HuggingFace spaces that can RAG 10-k reports and answer the questions of financial analysts swiftly.

The project will involve testing the RAG system on a current business problem. The Financial analysts are asked to research major cloud and AI platforms such as Amazon AWS, Google Cloud, Microsoft Azure, Meta AI, and IBM Watson to determine the most effective platform for this application. The primary goals include improving the efficiency of data extraction. Once the project is deployed, the system will be tested by a financial analyst with the following questions. Accurate text retrieval for these questions will imply the project's success.

**Questions:**

1. Has the company made any significant acquisitions in the AI space, and how are these acquisitions being integrated into the company's strategy?

2. How much capital has been allocated towards AI research and development?

3. What initiatives has the company implemented to address ethical concerns surrounding AI, such as fairness, accountability, and privacy?

4. How does the company plan to differentiate itself in the AI space relative to competitors?

Each Question must be asked for each of the five companies on the HuggingFace spaces.


**By successfully developing this project, we aim to:**

Improve the productivity of financial analysts by providing a competent tool.

Provide timely insights to improve client recommendations.

Strengthen FinTech Insights Inc.’s competitive edge by delivering more reliable and faster insights to clients.


**Connect to a T4 GPU Instance to create the Vector Database.**

### Setup

In [1]:
# Install the necessary libraries
!pip install -q openai==1.23.2 \
                tiktoken==0.6.0 \
                pypdf==4.0.1 \
                langchain==0.1.1 \
                langchain-community==0.0.13 \
                chromadb==0.4.22 \
                sentence-transformers==2.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import the necessary Libraries
import json
import tiktoken

import pandas as pd

from openai import OpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.vectorstores import Chroma

from google.colab import userdata, drive

# Impementing RAG

### Prepare Data

Let's start by loading the dataset.

In [4]:
#Upload Dataset-10k.zip and unzip it dataset folder using -d option
!unzip Dataset-10k.zip -d dataset

Archive:  Dataset-10k.zip
  inflating: dataset/IBM-10-k-2023.pdf  
  inflating: dataset/Meta-10-k-2023.pdf  
  inflating: dataset/aws-10-k-2023.pdf  
  inflating: dataset/google-10-k-2023.pdf  
  inflating: dataset/msft-10-k-2023.pdf  


## DB Creation

### Chunking

In [5]:
# Provide pdf_folder_location
pdf_folder_location = "dataset"

In [6]:
# Load the directory to pdf_loader
pdf_loader = PyPDFDirectoryLoader(pdf_folder_location)

In [10]:
# Create text_splitter using recursive splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name='cl100k_base',
    chunk_size=512,
    chunk_overlap=16
)

In [11]:
# Create chunks
report_chunks = pdf_loader.load_and_split(text_splitter)

In [13]:
# Check the total number of chunks
len(report_chunks)

908

In [14]:
# Check the first object in report_chunks and print it
report_chunks[0]

Document(page_content="UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________\nFORM 10-K  \n___________________________________________\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2023  \nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from              to             .\nCommission file number: 001-37580  \n___________________________________________\nAlphabet Inc.  \n(Exact name of registrant as specified in its charter)\n___________________________________________\nDelaware 61-1767919\n(State or other jurisdiction of incorporation or organization) (I.R.S. Employer Identification No.)\n1600 Amphitheatre Parkway  \nMountain View , CA 94043  \n(Address of principal executive offices, including zip code)\n(650) 253-0000  \n(Registrant's telephone number, inc

### Database Creation

In [15]:
#Create a Colelction Name
collection_name = 'report-10k-2024'

In [17]:
embedding_model_name='thenlper/gte-large'

In [18]:
# Initiate the embedding model 'thenlper/gte-large'
embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [19]:
# Create the vector Database
vectorstore = Chroma.from_documents(
    report_chunks,
    embedding_model,
    collection_name=collection_name,
    persist_directory='./dataset-10k'
)

In [20]:
# Persist the DB
vectorstore.persist()

In [21]:
vectorstore_persisted = Chroma(
    collection_name=collection_name,
    persist_directory='./dataset-10k',
    embedding_function=embedding_model
)

In [22]:
retriever = vectorstore_persisted.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

In [23]:
#Mount the Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
#Copy the persisted database to your drive
!cp -r dataset-10k /content/drive/MyDrive/

## Load Retriever with Vector DB from Google Drive

In [26]:
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Since we persisted the database to a Google Drive location, we can download the database to the instance using its unique id like so:

In [28]:
persisted_vectordb_location = '/content/drive/MyDrive/dataset-10k'

In practise, the database is maintained as a separate entity and CRUD operations are managed just as one would for normal databases (e.g., relational databases).

In [29]:
dataset_10k_collection = 'report-10k-2024'

In [30]:
vectorstore_persisted = Chroma(
    collection_name=dataset_10k_collection,
    persist_directory=persisted_vectordb_location,
    embedding_function=embedding_model
)

In [31]:
retriever = vectorstore_persisted.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

# Retrieve DB from GDrive

###**Set up CPU Instance**

In [32]:
# Install the required packages
!pip install -q openai==1.23.2 \
                tiktoken==0.6.0 \
                langchain==0.1.1 \
                langchain-community==0.0.13 \
                chromadb==0.4.22 \
                sentence-transformers==2.3.1

In [33]:
# Import the necessary Libraries
import json
import tiktoken

import pandas as pd

from openai import OpenAI

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.vectorstores import Chroma

from google.colab import userdata, drive

### Set up Anyscale Credentials

In [34]:
#get anyscale api key
anyscale_api_key = userdata.get('anyscale_api_key')

In [35]:
# Initialise the client
client = OpenAI(
    base_url="https://api.endpoints.anyscale.com/v1",
    api_key=anyscale_api_key
)

We are going to use Mixtral 8x7B model for this exercise due to it's higher performance.

In [38]:
#Provide the model name
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

### Mount Google Drive

In [39]:
#Mount the Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Since we persisted the database to a Google Drive location, we can download the database to the instance using its unique id like so:

### Load Vector DB from Google Drive

In [42]:
# Initialise the embedding model
embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')

In [43]:
# Load the persisted DB
persisted_vectordb_location = '/content/drive/MyDrive/dataset-10k'

In practise, the database is maintained as a separate entity and CRUD operations are managed just as one would for normal databases (e.g., relational databases).

In [44]:
#Create a Colelction Name
collection_name = 'report-10k-2024'

In [50]:
# Load the persisted DB
reports_db = Chroma(
    collection_name=collection_name,
    persist_directory=persisted_vectordb_location,
    embedding_function=embedding_model
)

### Test your DB

In [47]:
user_question = "How is the company integrating AI across their various business units, and what specific examples are provided in the reports of AI enhancing operational efficiencies or customer experiences?"

In [51]:
# Perform similarity search on the user_question
# You must add an extra parameter to the similarity search  function so that you can filter the response based on the 'source'  in the metadata of the doc
# The filter can be added as a parameter to the similarity search function
# This will allow you to retrieve chunks from a particular document
# Use the same format to filter your response based on the company.
docs = reports_db.similarity_search(user_question, k=5, filter = {"source":"dataset/google-10-k-2023.pdf"}) # Note the format to add a filter. You must apply the same in your app.py file that you will upload on huggingface spaces

In [53]:
# Print the retrieved docs, their source and the page number
# (page number can be accessed using doc.metadata['page'] )
for i, doc in enumerate(docs):
    print(f"Retrieved chunk {i+1}: \n")
    print(doc.page_content.replace('\t', ' '))
    print("Source: ", doc.metadata['page'],"\n ===================================================== \n")
    print('\n')

Retrieved chunk 1: 

Our business environment is rapidly evolving and intensely competitive. Our businesses face changing 
technologies, shifting user needs, and frequent introductions of rival products and services. To compete successfully, 
we must accurately anticipate technology developments and deliver innovative, relevant and useful products, services, 
and technologies in a timely manner. As our businesses evolve, the competitive pressure to innovate will encompass a 
wider range of products and services. We must continue to invest significant resources in technical infrastructure and 
R&D, including through acquisitions, in order to enhance our technology, products , and services . 
We have many competitors in different industries. Our current and potential domestic and international 
competitors range from large and established companies to emerging start-ups. Some competitors have longer 
operating histories and well-established relationships in various sectors. They can use 

In [82]:
retriever = reports_db.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

The vectorDB seems to work fine, let's move on to the next step.

## RAG Q&A

### Prompt Design

Let's formulate a prompt to fetch a Retrieval Augmented Generation from the LLM.

It is essential to ensure that the LLM does not generate hallucinated or random answers when faced with irrelevant information in the retrieved documents. Additionally, the LLM should provide the source of the information it offers. Including source citations will enhance the reliability and credibility of our chatbot.

In [55]:
# Create a system message for the LLM
qna_system_message = """
You are an assistant to a coder. Your task is to provide relevant information about the Python package Streamlit.

User input will include the necessary context for you to answer their questions. This context will begin with the token: ###Context.
The context contains references to specific portions of documents relevant to the user's query, along with source links.
The source for a context will begin with the token ###Source

When crafting your response:
1. Select only context relevant to answer the question.
2. Include the source links in your response.
3. User questions will begin with the token: ###Question.
4. If the question is irrelevant to streamlit respond with - "I am an assistant for streamlit Docs. I can only help you with questions related to streamlit"

Please adhere to the following guidelines:
- Your response should only be about the question asked and nothing else.
- Answer only using the context provided.
- Do not mention anything about the context in your final answer.
- If the answer is not found in the context, it is very very important for you to respond with "I don't know. Please check the docs @ 'https://docs.streamlit.io/'"
- Always quote the source when you use the context. Cite the relevant source at the end of your response under the section - Source:
- Do not make up sources. Use the links provided in the sources section of the context and nothing else. You are prohibited from providing other links/sources.

Here is an example of how to structure your response:

Answer:
[Answer]

Source:
[Source]
"""

In [56]:
# Create a message template
qna_user_message_template = """
###Context
Here are some documents and their source links that are relevant to the question mentioned below.
{context}

###Question
{question}
"""

### Composing the response

In [59]:
# Create a variable company to store the source of the context so that you can filter the similarity search
company = "dataset/aws-10-k-2023.pdf"

In [60]:
# Fetch relevant documents and create context for query by joining page_content and page number of the retrieved docs
relevant_document_chunks = retriever.get_relevant_documents(company)
context_list = [d.page_content for d in relevant_document_chunks]
context_for_query = ". ".join(context_list)

print(context_for_query) # Print the whole context_for_query (after joining all the chunks. It should contain page number of every chunk)

Table of Contents
Results of Operations
We have organized our operations into three segments: North America, International, and AWS. These segments reflect the way the Company evaluates
its business performance and manages its operations. See Item 8 of Part II, “Financial Statements and Supplementary Data — Note 10 — Segment
Information.”
Overview
Macroeconomic factors, including inflation, increased interest rates, significant capital market volatility, the prolonged COVID-19 pandemic, global
supply chain constraints, and global economic and geopolitical developments, have direct and indirect impacts on our results of operations that are difficult to
isolate and quantify. These factors contributed to increases in our operating costs during 2022, particularly across our North America and International
segments, primarily due to a return to more normal, seasonal demand volumes in relation to our fulfillment network fixed costs, increased transportation and
utility costs, and increased w

In [62]:
# Craft the messages to pass to chat.completions.create
prompt = [
    {'role':'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=company
        )
    }
]

In [64]:
# Get a response from the LLM
# Handle errors using try-except
# print the content of the response


response = client.chat.completions.create(
    model=model_name,
    messages=prompt,
    temperature=0
)

answer = response.choices[0].message.content.strip()

print(answer)

Answer:
The AWS segment consists of amounts earned from global sales of compute, storage, database, and other services for start-ups, enterprises, government agencies, and academic institutions.

Source:
- Note 10 — SEGMENT INFORMATION
- We have organized our operations into three segments: North America, International, and AWS. The AWS segment consists of amounts earned from global sales of compute, storage, database, and other services for start-ups, enterprises, government agencies, and academic institutions.
- https://docs.streamlit.io/en/stable/streamlit_website_docs/streamlit_website/top_level_pages/about_us/company/financials/aws_10k_2023.pdf#page=11


# Evaluation

### Craft prompts for evaluation

In [65]:
# Pick a model that's offers more performace as a rater_model. Most of the time a model with more parameters is more performant.
rater_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [66]:
# Create a prompt for the rater LLM to check the groundedness of the response
groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
"""

In [67]:
# Create a prompt for the rater LLM to check the relevance of the response
relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
"""

In [68]:
#Create user message template such that question, answer and context can be provided through it.
user_message_template = """
###Question
{question}

###Context
{context}

###Answer
{answer}
"""

### Test the Evaluation on One Sample

In [70]:
user_input = "How much is the company investing in research and development, and what are the key areas of focus for innovation?"

In [71]:
# Fetch relevant documents and create context for query by joining page_content and page number of the retrieved docs
relevant_document_chunks = retriever.get_relevant_documents(user_input)
context_list = [d.page_content for d in relevant_document_chunks]
context_for_query = ". ".join(context_list)

In [72]:
# Create the messages for chat.completion.create()
prompt = [
    {'role':'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=user_input
        )
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=prompt,
    temperature=0
)

In [75]:
# Get a response from the LLM
# Handle errors using try-except
answer = response.choices[0].message.content.strip()

print(answer)

Answer:
The company is investing a total of $88.15 billion in research and development for the year 2023. The key areas of focus for innovation include:

1. Cloud and AI: Developing Azure AI platform and cloud infrastructure, server, database, CRM, ERP, software development tools and services, AI cognitive services, and other business process applications and services for enterprises.
2. Strategic Missions and Technologies: Incubating technical products and support solutions with transformative potential for the future of cloud computing and continued company growth across quantum computing, Azure Space & Missions Engineering, telecommunications, and Microsoft Federal Sales and Delivery.
3. Experiences and Devices: Delivering high-value end-user experiences across products, services, and devices, including Microsoft 365, Windows, Microsoft Teams, Search (including Microsoft Edge and Bing Chat) and other advertising-based services, and the Surface line of devices.
4. Microsoft Security:

In [76]:
# Create messages for groundness LLM
groundedness_prompt = [
    {'role':'system', 'content': groundedness_rater_system_message},
    {'role': 'user', 'content': user_message_template.format(
        question=user_input,
        context=context_for_query,
        answer=answer
        )
    }
]

In [77]:
# Print the response of the rater LLM on groundedness
response = client.chat.completions.create(
    model=rater_model,
    messages=groundedness_prompt,
    temperature=0
)

print(response.choices[0].message.content)

 1. Steps to evaluate the answer:
   a. Identify the key points in the answer and match them with the context provided.
   b. Check if the answer mentions the amount of investment in research and development.
   c. Verify if the answer lists the key areas of focus for innovation as mentioned in the context.
   d. Confirm if the answer is derived only from the information presented in the context.

2. Step-by-step explanation of the answer adhering to the metric:
   a. The answer states that the company is investing a total of $88.15 billion in research and development for the year 2023, which matches the context.
   b. The answer then lists the key areas of focus for innovation, which are all mentioned in the context.
   c. The answer does not include any information that is not present in the context.

3. Extent to which the metric is followed: The metric is followed completely, as the answer is derived only from the information presented in the context.

4. Evaluation score: 5 (The m

In [78]:
# Print the response of the rater LLM on relevance
relevance_prompt = [
    {'role':'system', 'content': relevance_rater_system_message},
    {'role': 'user', 'content': user_message_template.format(
        question=user_input,
        context=context_for_query,
        answer=answer
        )
    }
]

In [79]:
# Print the response of the rater LLM on relevance
response = client.chat.completions.create(
    model=rater_model,
    messages=relevance_prompt,
    temperature=0
)

print(response.choices[0].message.content)

 1. Steps to evaluate the context as per the relevance metric:
- Identify the main aspects of the question:


### Evaluation on multiple-queries

In [83]:
# List of queries
queries = [ "What are the company’s policies and frameworks regarding AI ethics, governance, and responsible AI use as detailed in their 10-K reports?",
           "What are the primary business segments of the company, and how does each segment contribute to the overall revenue and profitability?",
            "What are the key risk factors identified in the 10-K report that could potentially impact the company’s business operations and financial performance?"

]
# Create a DataFrame to store the results
df = pd.DataFrame(columns=['query', 'response', 'context', 'groundedness_evaluation', 'relevance_evaluation'])

# run a loop to get answer for every query and every company and then rate them on groundedness and relevance
# store the query, response, context,groundedness_evaluation, relevance_evaluation in a dataframe

for query in queries:


    relevant_document_chunks = retriever.get_relevant_documents(query)
    #context_list = [d.page_content + "\n ###Source: " + d.metadata['page'] + "\n\n " for d in relevant_document_chunks]
    context_list = [d.page_content for d in relevant_document_chunks]

    context_for_query = ". ".join(context_list)

    prompt = [
        {'role':'system', 'content': qna_system_message},
        {'role': 'user', 'content': qna_user_message_template.format(
            context=context_for_query,
            question=query
            )
        }
    ]

    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=0
    )

    answer = response.choices[0].message.content.strip()
    # print(context_for_query)
    # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # print(answer)
    # print("====================================================================")

    # Groundedness prompt
    groundedness_prompt = [
        {'role':'system', 'content': groundedness_rater_system_message},
        {'role': 'user', 'content': user_message_template.format(
            question=query,
            context=context_for_query,
            answer=answer
            )
        }
    ]

    # Get the groundedness response
    response = client.chat.completions.create(
        model=rater_model,
        messages=groundedness_prompt,
        temperature=0
    )
    groundedness_response = response.choices[0].message.content

    # Relevance prompt
    relevance_prompt = [
        {'role':'system', 'content': relevance_rater_system_message},
        {'role': 'user', 'content': user_message_template.format(
            question=query,
            context=context_for_query,
            answer=answer
            )
        }
    ]

    # Get the relevance response
    response = client.chat.completions.create(
        model=rater_model,
        messages=relevance_prompt,
        temperature=0
    )
    relevance_response = response.choices[0].message.content

    # Store the query and responses in the DataFrame
    df = pd.concat([df, pd.DataFrame([{'query': query,'response': answer, 'context': context_for_query, 'groundedness_evaluation': groundedness_response, 'relevance_evaluation': relevance_response}])], ignore_index=True)
df.head(13)

,query,response,context,groundedness_evaluation,relevance_evaluation
0,What are the company’s policies and frameworks...,Answer:\nThe company has a commitment to respo...,5 to launch a Generative AI Skills Grant Chall...,"1. To evaluate the answer, I would follow the...",1. The steps to evaluate the context as per t...
1,What are the primary business segments of the ...,Answer:\nThe primary business segments of the ...,of our key businesses. The segments enable the...,1. The steps to evaluate the answer based on ...,1. Steps to evaluate the context as per the r...
2,What are the key risk factors identified in th...,Answer:\nThe key risk factors identified in the 1,Table of Contents\nItem 1A.Risk Factors\nCerta...,"1. First, read through the entire context and...",1. To evaluate the context as per the relevan...


In [84]:
# Your Dataframe should have 15 rows - 3 queries for each of 5 companies - 3*5 = 15
# Show the top 10 rows of the dataframe
df.head(10)

,query,response,context,groundedness_evaluation,relevance_evaluation
0,What are the company’s policies and frameworks...,Answer:\nThe company has a commitment to respo...,5 to launch a Generative AI Skills Grant Chall...,"1. To evaluate the answer, I would follow the...",1. The steps to evaluate the context as per t...
1,What are the primary business segments of the ...,Answer:\nThe primary business segments of the ...,of our key businesses. The segments enable the...,1. The steps to evaluate the answer based on ...,1. Steps to evaluate the context as per the r...
2,What are the key risk factors identified in th...,Answer:\nThe key risk factors identified in the 1,Table of Contents\nItem 1A.Risk Factors\nCerta...,"1. First, read through the entire context and...",1. To evaluate the context as per the relevan...


You might experience some hallucination in LLM's response. Try to change your prompt to mitigate this. Selecting a good model will also help mitigating hallucination, increase groundedness and relevance.

# Gradio Interface

In [ ]:
%%writefile app.py


## Setup
# Import the necessary Libraries
import os
import uuid
import json

import gradio as gr

from openai import OpenAI

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

from huggingface_hub import CommitScheduler
from pathlib import Path
from dotenv import load_dotenv


# Create Client
load_dotenv()

os.environ["ANYSCALE_API_KEY"]=os.getenv("ANYSCALE_API_KEY")

client = OpenAI(
    base_url="https://api.endpoints.anyscale.com/v1",
    api_key=os.environ['ANYSCALE_API_KEY']
)

embedding_model = SentenceTransformerEmbeddings(model_name='thenlper/gte-large')
# Define the embedding model and the vectorstore

collection_name = 'report-10k-2024'

vectorstore_persisted = Chroma(
    collection_name=collection_name,
    persist_directory='./dataset-10k',
    embedding_function=embedding_model
)

# Load the persisted vectorDB

retriever = vectorstore_persisted.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)


# Prepare the logging functionality

log_file = Path("logs/") / f"data_{uuid.uuid4()}.json"
log_folder = log_file.parent

scheduler = CommitScheduler(
    repo_id="---------",
    repo_type="dataset",
    folder_path=log_folder,
    path_in_repo="data",
    every=2
)

# Define the Q&A system message

qna_system_message = """
You are an assistant to a coder. Your task is to provide relevant information about the Python package Streamlit.

User input will include the necessary context for you to answer their questions. This context will begin with the token: ###Context.
The context contains references to specific portions of documents relevant to the user's query, along with source links.
The source for a context will begin with the token ###Source

When crafting your response:
1. Select the most relevant context or contexts to answer the question.
2. Include the source links in your response.
3. User questions will begin with the token: ###Question.
4. If the question is irrelevant to streamlit respond with - "I am an assistant for streamlit Docs. I can only help you with questions related to streamlit"

Please adhere to the following guidelines:
- Answer only using the context provided.
- Do not mention anything about the context in your final answer.
- If the answer is not found in the context, it is very very important for you to respond with "I don't know. Please check the docs @ 'https://docs.streamlit.io/'"
- Always quote the source when you use the context. Cite the relevant source at the end of your response under the section - Sources:
- Do not make up sources. Use the links provided in the sources section of the context and nothing else. You are prohibited from providing other links/sources.

Here is an example of how to structure your response:

Answer:
[Answer]

Source
[Source]
"""

# Define the user message template
qna_user_message_template = """
###Context
Here are some documents that are relevant to the question.
{context}
```
{question}
```
"""

# Define the predict function that runs when 'Submit' is clicked or when a API request is made
def predict(user_input,company):

    filter = "dataset/"+company+"-10-k-2023.pdf"
    relevant_document_chunks = vectorstore_persisted.similarity_search(user_input, k=5, filter={"source":filter})

    # Create context_for_query
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ".".join(context_list)

    # Create messages
    prompt = [
        {'role':'system', 'content': qna_system_message},
        {'role': 'user', 'content': qna_user_message_template.format(
            context=context_for_query,
            question=user_input
            )
        }
    ]

    # Get response from the LLM
    try:
        response = client.chat.completions.create(
            model='mistralai/Mixtral-8x7B-Instruct-v0.1',
            messages=prompt,
            temperature=0
        )

        prediction = response.choices[0].message.content

    except Exception as e:
        prediction = e

    # While the prediction is made, log both the inputs and outputs to a local log file
    # While writing to the log file, ensure that the commit scheduler is locked to avoid parallel
    # access

    with scheduler.lock:
        with log_file.open("a") as f:
            f.write(json.dumps(
                {
                    'user_input': user_input,
                    'retrieved_context': context_for_query,
                    'model_response': prediction
                }
            ))
            f.write("\n")

    return prediction

# Set-up the Gradio UI
# Add text box and radio button to the interface
# The radio button is used to select the company 10k report in which the context needs to be retrieved.

textbox = gr.Textbox()
company = gr.Radio()

# Create the interface
# For the inputs parameter of Interface provide [textbox,company]


demo.queue()
demo.launch()

### Paste your gradio app link and logs link

*   app link here

*   logs_dataset link here

Note: Make sure your Hugging Face space repository and the logs_dataset are set to public. If it's private, the evaluator won't be able to access the app you've built, which could result in losing marks.

# Convert ipynb to HTML

Instructions:
1. Go to File
2. Download these current working Notebook in to ipynb format
3. Now, run the below code, select the notebook from local where you downloaded the file
4. Wait for few sec, your notebook will automatically converted in to html format and save in your local pc


In [ ]:
# @title HTML Convert
# Upload ipynb
from google.colab import files
f = files.upload()

# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip", "install", "nbconvert"])
_ = subprocess.run(["jupyter", "nbconvert", file0, "--to", "html"])

# download the html
files.download(file0[:-5]+"html")


## Power Ahead!